# Getty TGN | check on longitudes and latitudes by cross-checking other sources
- 2024-05-18
- Geo check on Getty TGN data fetched via SPARQL-endpoint
- https://amandinancy16.medium.com/reverse-geocoding-with-geopy-c26cfb63f74c
- V. Martens

# Import

## Import packages

In [1]:
# creating time stamps
import time

# importing files
import glob
import os

# progress bar
# from tqdm.notebook import tqdm
from tqdm import tqdm

# regex module
import re

# for multi-threading
from concurrent import futures
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp

# retrieve country names from country codes
import pycountry

# data wrangling
import pandas as pd
import numpy as np

# back up files
import pickle

# retrieve various geo locations
import reverse_geocoder as rg
from geopy.geocoders import Nominatim

# measure distance between two geo points
from haversine import haversine, Unit

# save and dump
import pickle

# surpress, warnings, uhoohhhh
import warnings
warnings.filterwarnings("ignore")

# preferences
# adjust pandas to show all cols
pd.set_option('display.max_colwidth', None)

## Import functions

In [3]:
def load_latest_file(filepath:str) -> str:
    '''
    loads created latest create file from a directory
    args: string with filepath
    returns: latest file from a list of files
    '''

    list_of_files = glob.glob(filepath)
    latest_file = max(list_of_files, key=os.path.getctime)
    print(latest_file)
    
    return latest_file

def join_lats_lons(df:pd.DataFrame) -> pd.DataFrame:
    '''
    args: pd.DataFrame with lats and lons
    returns: pd.DataFrame with corrected lats and lons and joined into one col 
    '''
    
    df['lat'] = df['lat'].str.replace('-.','-0.')
    df['lon'] = df['lon'].str.replace('-.','-0.')
    df['coordinates'] = list(zip(df['lat'], df['lon']))
    
    return df

def get_country(df:pd.DataFrame, i:int) -> list:

    '''
    args: pd.DataFrame() with coordinates column
    returns: a list with dicts with geo locations, every dict contains keys with country name, 3 admin names, lat, lon
    '''
    
    results = rg.search(df['coordinates'].iloc[i])

    for item in results:
        query = df['coordinates'].iloc[i]
        tgn_id = df['tgn_id'].iloc[i]
        country = item['cc']
        looked_lat = item['lat']
        looked_lon = item['lon']
        looked_name = item['name']
        
    return [tgn_id, query, country, looked_lat, looked_lon, looked_name]

def parse_reverse_geo_lookup(list_of_dicts:list) -> pd.DataFrame:
    '''
    args: a list with dicts with geo locations
    returns: pd.DataFrame() with parsed geo locations
    '''
    
    dfs = pd.DataFrame()

    for coord_dict in list_of_dicts:
        df = pd.DataFrame(coord_dict).T
        dfs = pd.concat([dfs, df])
    
    dfs = dfs.rename(columns={0 : 'tgn_id_lookup',
                              1 : 'query_look_up',
                              2 : 'country_code_lookup', 
                              3 : 'lat_lookup',
                              4 : 'lon_lookup', 
                              5 : 'municipality_lookup'})
    
    return dfs
    
def parse_parentstring(df:pd.DataFrame) -> pd.DataFrame:
    '''
    parses out info from parentstring
    args: pd.DataFrame() with tgn parentstrings
    returns: pd.DataFrame() with reordered and parsed columns
    '''
    
    df['parentstring_province'] = (df['parentstring']
                                    .str.split(',')
                                    .str[0]
                                    .str.strip())
    
    df['parentstring_country'] = (df['parentstring']
                                    .str.split(',')
                                    .str[1]
                                    .str.strip())
    
    df['parentstring_continent'] = (df['parentstring']
                                    .str.split(',')
                                    .str[2]
                                    .str.strip())  
    
    df['parentstring_world'] = (df['parentstring']
                                    .str.split(',')
                                    .str[3]
                                    .str.strip())  
    
    df = df[['tgn_id', 'city_name', 'inferred_city_name', 'parentstring_province',
             'parentstring_country', 'parentstring_continent', 'parentstring_world',
             'broader_parentstring', 'lat', 'lon', 'coordinates', 'query_look_up',
             'country_code_lookup', 'lat_lookup', 'lon_lookup',
             'municipality_lookup']]
    
    return df    

def parse_broader_parentstring(df:pd.DataFrame) -> pd.DataFrame:
    '''
    parses out info from broader parentstring
    args: pd.DataFrame() with tgn broader parentstrings
    returns: pd.DataFrame() with reordered and parsed columns
    '''
    
    df['broader_parentstring_country'] = (df['broader_parentstring']
                                    .str.split()
                                    .str[0]
                                    .str.replace(',','')
                                    .str.strip())
    
    df['broader_parentstring_continent'] = (df['broader_parentstring']
                                    .str.split()
                                    .str[1]
                                    .str.replace(',','')
                                    .str.strip())  
    
    df['broader_parentstring_world'] = (df['broader_parentstring']
                                    .str.split()
                                    .str[2]
                                    .str.replace(',','')
                                    .str.strip())  
    
    df = df[['tgn_id', 'city_name', 'inferred_city_name', 'parentstring_province',
       'parentstring_country', 'parentstring_continent', 'parentstring_world',
       'broader_parentstring_country', 'broader_parentstring_continent',
       'broader_parentstring_world', 'lat', 'lon', 'coordinates', 'query_look_up',
       'country_code_lookup', 'lat_lookup', 'lon_lookup', 'municipality_lookup']]
    
    return df

def country_code_lookup(country_code:str) -> str:
    '''
    
    retrieves countryname from countrycode
    args: string with countrycode
    returns: returns country name
    '''
    return pycountry.countries.get(alpha_2=country_code).name

def municipality_look_up(df:pd.DataFrame) -> dict:
    '''
    pd.DataFrame with city names retrieves city info, lats, lons
    args: pd.DataFrame with city name/municipalities
    returns: a dict with city names as keys and location info and lats, lons as values
    '''
    geolocator = Nominatim(user_agent="Nancy Amandi", timeout=10)
    
    dict_locs = {}
    
    for i in tqdm(range(len(df)), total=len(df), desc='find geo info based on city names'):
        dict_locs[df['municipality_lookup'].iloc[i]] = geolocator.geocode(df['municipality_lookup'].iloc[i])
    
    return dict_locs

def parse_geo_lookup(results_lookup:dict) -> pd.DataFrame:
    '''
    parses looked up geo data into a pd.DataFrame
    args: a dict with looked up geo locations based on city names
    returns: a pd.DataFrame with city names, lat-lons
    '''
    df_parsed_geo = (pd.DataFrame(results_lookup)
                    .T.reset_index()
                    .rename(columns={'index':'query_city_name',
                                    0 : 'query_city_city_info',
                                    1 : 'query_city_geo_data'}))
    
    return df_parsed_geo

def data_haversine_tuple_creator(df:pd.DataFrame) -> pd.DataFrame:
    '''
    args: pd.DataFrame with seperate lat lon
    returns: pd.Dataframe with a tuple of lat lon in one col
    '''
    
    df['coordinates_looked_up'] = list(zip(df['lat_lookup'].astype(float), df['lon_lookup'].astype(float)))
    df = df.drop(columns=['lat_lookup', 'lon_lookup'])
    
    return df

def haversine_distance_calculator(df) -> pd.DataFrame:
    '''
    measures distance between two tuples containing two geo points each 
    args: a pd.DataFrame with two columns containing tuples with floats
    returns: a pd.DataFrame with a column that has measured the difference in kms between two tuples with geo points
    '''
    df['km_difference'] = '' 
    
    for i in tqdm(range(len(df)), total=len(df), desc='find geo info based on city names'):
        df['km_difference'].iloc[i] = round(haversine(df['query_city_geo_data'].iloc[i], df['coordinates_looked_up'].iloc[i], unit=Unit.KILOMETERS),2)
        
    return df

## Constants

In [4]:
# create back up filename for a pickle
time_stamp = time.strftime('%Y%m%d-%H%M%S')
filename_df_errors = f'{time_stamp}_df_errors_tgn.pickle'

# create back up filename for a pickle
time_stamp = time.strftime('%Y%m%d-%H%M%S')
filename_df_lod_results = f'{time_stamp}_df_lod_results_tgn.pickle'

# create back up filename for a pickle
time_stamp = time.strftime('%Y%m%d-%H%M%S')
filename_excel_export = f'{time_stamp}_loc_check_TGN.xlsx'

print(f"{filename_df_errors}, {filename_df_lod_results}, {filename_excel_export}")

20240607-143032_df_errors_tgn.pickle, 20240607-143032_df_lod_results_tgn.pickle, 20240607-143032_loc_check_TGN.xlsx


## Import data

In [5]:
latest_picke_file = load_latest_file('data_dumps/*tgn_country.pickle')

# Open the file in binary mode
with open(latest_picke_file, 'rb') as file:
      
    # Call load method to deserialze
    df_tgn = pickle.load(file)
    
# enrich lats, lons
df_tgn = join_lats_lons(df_tgn)
df_tgn.shape



data_dumps\results_df_tgn_country.pickle


# Subset data

In [ ]:
# subset data
df_nl = df_tgn[(df_tgn['broader_parentstring'].str.contains('Netherlands') == True)]
df_europe = df_tgn[(df_tgn['broader_parentstring'].str.contains('Europe') == True)]
df_nl.shape, df_europe.shape

# Geo data-check
 1. Reverse city look-up on geo lats + lons 
 1. Retrieve lats, lons based on reversed look-up
 1. Compare distances between two geo-points

In [7]:
%%time

import multiprocessing as mp
pools = mp.cpu_count()

# Start pool
thread_pool = ThreadPoolExecutor(max_workers=pools, thread_name_prefix = 'thread')

# reate futures
futures = [thread_pool.submit(get_country, df_nl, i) for i in range(len(df_nl))]

# submit tasks
results = [future.result() for future in tqdm(futures, total=len(futures), desc='find reverse geo codes')]

# reverse geo lookup to a df
df_geo_data = parse_reverse_geo_lookup(results)

# merge results geo lookop with main tgn dataset
df_nl = (pd.merge(df_nl, df_geo_data, left_on='tgn_id', right_on='tgn_id_lookup', how='left')
         .drop(columns=['tgn_id_lookup'])
         )

# parse tgn dataset into seperate cols
df_nl = parse_parentstring(df_nl)
df_nl = parse_broader_parentstring(df_nl)

# change country code into country name to allign with tgn data
df_nl['country_name_lookup'] = (df_nl['country_code_lookup']
                                .apply(country_code_lookup)
                                )

del df_nl['country_code_lookup']

# look up lat lons based on reverse geo lookup and return lat lons
results_lookup = municipality_look_up(df_nl)

# parse geo lookup into df
df_results = parse_geo_lookup(results_lookup)

# merge geo lookup with main tgn dataset
df_nl = (pd.merge(df_nl, df_results, left_on='municipality_lookup', right_on='query_city_name', how='left')
         .drop(columns=['query_city_name'])
         )

# creates a col with coordinate tuples for haversine distance measurements
df_nl = data_haversine_tuple_creator(df_nl)

# measures differences between two geo locations
df_nl = haversine_distance_calculator(df_nl)

df_nl.head()

Loading formatted geocoded file...
Loading formatted geocoded file...
Loading formatted geocoded file...
Loading formatted geocoded file...
Loading formatted geocoded file...
Loading formatted geocoded file...
Loading formatted geocoded file...
Loading formatted geocoded file...


find reverse geo codes:  20%|██        | 398/1947 [14:35<45:09,  1.75s/it]  

In [30]:
df_nl['lat_diff'] = ''
for i in tqdm(range(len(df_nl)), total=len(df_nl), desc='find geo info based on city names'):
    df_nl['lat_diff'].iloc[i] = float(df_nl['query_look_up'][i][0]) - float(df_nl['query_city_geo_data'][i][0])

with open(f'data_dumps/{filename_excel_export}.pickle', 'wb') as handle:
    pickle.dump(df_nl, handle, protocol=pickle.HIGHEST_PROTOCOL)

find geo info based on city names: 100%|██████████| 50/50 [00:00<00:00, 1704.16it/s]


In [43]:
df_nl[(df_nl['km_difference'] > 5) & (df_nl['query_look_up'].astype(str).str.contains('52') == False)]


,tgn_id,city_name,inferred_city_name,parentstring_province,parentstring_country,parentstring_continent,parentstring_world,broader_parentstring_country,broader_parentstring_continent,broader_parentstring_world,...,lon,coordinates,query_look_up,municipality_lookup,country_name_lookup,query_city_city_info,query_city_geo_data,coordinates_looked_up,km_difference,lat_diff
5,http://vocab.getty.edu/tgn/7270017,Aabeek,Limburg,Limburg,Netherlands,Europe,World,Netherlands,Europe,World,...,6,"(51.25, 6)","(51.25, 6)",Leeuwen,Netherlands,"Leeuwen, Wageningen-Hoog, Wageningen, Gelderland, Nederland, 6704 AN, Nederland","(51.9826484, 5.6758503)","(51.21032, 5.99862)",88.73,-0.732648
26,http://vocab.getty.edu/tgn/7006756,Arnemuiden,Zuid Beveland,Zuid Beveland,Zeeland,Netherlands,Europe,Zeeland,Netherlands,Europe,...,3.666667,"(51.5, 3.666667)","(51.5, 3.666667)",Veere,Netherlands,"Veere, Zeeland, Nederland","(51.55640385, 3.577269355747127)","(51.54833, 3.66667)",6.25,-0.056404


# Check cities and their names

In [ ]:
haversine on zip(lat_lookup, lon_lookup) and query_city_geo_data

In [29]:
df_nl.head()

,tgn_id,city_name,inferred_city_name,parentstring_province,parentstring_country,parentstring_continent,parentstring_world,broader_parentstring_country,broader_parentstring_continent,broader_parentstring_world,lat,lon,coordinates,query_look_up,lat_lookup,lon_lookup,municipality_lookup,country_name_lookup,query_city_city_info,query_city_geo_data
0,http://vocab.getty.edu/tgn/7257647,Aagtekerke,Zeeland,Zeeland,Netherlands,Europe,World,Netherlands,Europe,World,51.55,3.516667,"(51.55, 3.516667)","(51.55, 3.516667)",51.5,3.61389,Middelburg,Netherlands,"Middelburg, Zeeland, Nederland","(51.4996784, 3.6137371)"
1,http://vocab.getty.edu/tgn/7257816,Aardam,South Holland,South Holland,Netherlands,Europe,World,Netherlands,Europe,World,52.166667,4.766667,"(52.166667, 4.766667)","(52.166667, 4.766667)",52.15083,4.77639,Nieuwkoop,Netherlands,"Nieuwkoop, Zuid-Holland, Nederland","(52.1500947, 4.7761706)"
2,http://vocab.getty.edu/tgn/7257652,Aa,Utrecht,Utrecht,Netherlands,Europe,World,Netherlands,Europe,World,52.166667,4.983333,"(52.166667, 4.983333)","(52.166667, 4.983333)",52.17417,5.00139,Breukelen,Netherlands,"Breukelen, Stichtse Vecht, Utrecht, Nederland","(52.171959, 5.002675)"
3,http://vocab.getty.edu/tgn/7257651,Aadorp,Overijssel,Overijssel,Netherlands,Europe,World,Netherlands,Europe,World,52.366667,6.633333,"(52.366667, 6.633333)","(52.366667, 6.633333)",52.35667,6.6625,Almelo,Netherlands,"Almelo, Overijssel, Nederland","(52.3567956, 6.66725277634109)"
4,http://vocab.getty.edu/tgn/7257653,Aa,North Brabant,North Brabant,Netherlands,Europe,World,Netherlands,Europe,World,51.65,5.316667,"(51.65, 5.316667)","(51.65, 5.316667)",51.65333,5.2875,Vught,Netherlands,"Vught, Noord-Brabant, Nederland","(51.6511806, 5.23680014410728)"


52.1500947 4.7761706


In [47]:
import concurrent.futures
import multiprocessing as mp

from functools import partial

cpu_count_laptop = mp.cpu_count()

geolocator = Nominatim(user_agent="Nancy Amandi", timeout=10)
geocode = RateLimiter(geolocator, swallow_exceptions=True, min_delay_seconds=0.1, return_value_on_exception=None) 

with concurrent.futures.ThreadPoolExecutor(max_workers=cpu_count_laptop, thread_name_prefix='thread') as e:
    locations = list(e.map(partial(geocode, language='en', exactly_one=True), tqdm(list(set(df_nl["municipality_lookup"]))), chunksize=100))
 

100%|██████████| 5/5 [00:00<00:00, 1670.64it/s]


TypeError: 'Nominatim' object is not callable

In [46]:
import pandas as pd
# df = pd.DataFrame({'name': ['paris', 'berlin', 'london']})

tqdm.pandas()

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter




geolocator = Nominatim(user_agent="geopy.geocoders.Nominatim")
geocode = RateLimiter(geolocator, min_delay_seconds=1)


# df_nl['new_location'] = df_nl['municipality_lookup'].progress_apply(geocode)


# geolocator = Nominatim(user_agent="Nancy Amandi", timeout=10)
# geocode = RateLimiter(geolocator.reverse, min_delay_seconds=0.1)


df_nl['new_location'] = df_nl['municipality_lookup'].apply(geocode, language='en', exactly_one=True )




TypeError: 'Nominatim' object is not callable

In [49]:
df_europe['lat'] = df_europe['lat'].astype(str)

In [17]:
df_europe = df_europe[(df_europe['broader_parentstring'] != 'Europe, World')]



df_europe = df_europe[(df_europe['lat'].str.len() > 3)]

In [18]:
df = df_europe.head(50)
df

,tgn_id,city_name,inferred_city_name,parentstring,broader_parentstring,lat,lon,coordinates
70,http://vocab.getty.edu/tgn/5002851,Arcetri,Florence,"Florence, Firenze, Tuscany, Italy, Europe, World","Firenze, Tuscany, Italy, Europe, World",43.75,11.233333,"43.75, 11.233333"
114,http://vocab.getty.edu/tgn/1003911,Acilia,Rome,"Rome, Roma, Lazio, Italy, Europe, World","Roma, Lazio, Italy, Europe, World",41.7833,12.3667,"41.7833, 12.3667"
238,http://vocab.getty.edu/tgn/7011167,Kefallinía,Ionian Islands,"Ionian Islands, Greece, Europe, World","Greece, Europe, World",38.25,20.5,"38.25, 20.5"
264,http://vocab.getty.edu/tgn/7348001,"Abejuela, Rambla de",Valencia,"Valencia, Valencian Community, Spain, Europe, World","Valencian Community, Spain, Europe, World",39.816667,-0.916667,"39.816667, -0.916667"
266,http://vocab.getty.edu/tgn/7000462,Melilla,Melilla,"Melilla, Spain, Europe, World","Spain, Europe, World",35.316667,-2.95,"35.316667, -2.95"
267,http://vocab.getty.edu/tgn/1011565,"Almina, Punta",Ceuta,"Ceuta, Spain, Europe, World","Spain, Europe, World",35.900833,-5.277222,"35.900833, -5.277222"
268,http://vocab.getty.edu/tgn/7348121,Abáigar,Navarra,"Navarra, Navarre, Spain, Europe, World","Navarre, Spain, Europe, World",42.65,-2.183333,"42.65, -2.183333"
269,http://vocab.getty.edu/tgn/7348477,"Addaya, Islas de",Illes Balears,"Illes Balears, Islas Baleares, Comunidad Autónoma de las, Spain, Europe, World","Islas Baleares, Comunidad Autónoma de las, Spain, Europe, World",40.016667,4.2,"40.016667, 4.2"
359,http://vocab.getty.edu/tgn/7002748,Kardhítsa,Thessaly,"Thessaly, Greece, Europe, World","Greece, Europe, World",39.333333,21.75,"39.333333, 21.75"
365,http://vocab.getty.edu/tgn/8712281,Acidava,Olt,"Olt, Romania, Europe, World","Romania, Europe, World",44.378096,24.264647,"44.378096, 24.264647"


In [58]:
def eval_results(x):
    try:
        return (x.latitude, x.longitude)
    except:
        return (None, None)

In [19]:
import concurrent.futures
import multiprocessing as mp

from functools import partial

cpu_count_laptop = mp.cpu_count()

geolocator = Nominatim(user_agent="Nancy Amandi", timeout=10)
geocode = RateLimiter(geolocator.reverse, swallow_exceptions=True, min_delay_seconds=0.1, return_value_on_exception=None) 

with concurrent.futures.ThreadPoolExecutor(max_workers=cpu_count_laptop, thread_name_prefix='thread') as e:
    locations = list(e.map(partial(geocode, language='en', exactly_one=True), tqdm(list(set(df_europe["coordinates"]))), chunksize=100))
 

100%|██████████| 69553/69553 [00:01<00:00, 37806.55it/s]


ValueError: Must be a coordinate pair or Point

In [14]:
from pprint import pprint
for i in range(len(locations)):
    # pprint(locations[i].raw)
    print(locations[i].raw['address']['country'])
    # print(locations[i].raw['address']['city'])

Italy
Spain
Greece
Italy
Italy


In [15]:
locations


[Location(Via Daniele da Samarate, Acilia Sud, Municipio Roma X, Acilia, Rome, Roma Capitale, Lazio, 00125, Italy, (41.783444700000004, 12.366633893971906, 0.0)),
 Location(GR-7, Chelva, La Serranía, Valencia, Valencian Community, Spain, (39.81682133171245, -0.9169225999114939, 0.0)),
 Location(Δειλινάτα - Αγία Ευφημία, Deilinata, Argostoli Municipality, Kefallonia Regional Unit, Ioanian Islands, Peloponnese, Western Greece and the Ionian, 281 00, Greece, (38.24980475690789, 20.500489568256107, 0.0)),
 Location(6d, Via Bernardino Poccetti, Due Strade, Quartiere 3, Galluzzo, Florence, Tuscany, 50124, Italy, (43.7496143, 11.2326789, 0.0)),
 Location(La Salumeria, 24, Via del Banco di Santo Spirito, Ponte, Municipio Roma I, Rome, Roma Capitale, Lazio, 00193, Italy, (41.9000048, 12.4667571, 0.0))]

In [49]:
df_tgn['address'] = df_tgn.progress_apply(lambda row: geocode((row['lat.value'], row['long.value']), language='en', exactly_one=True), axis=1)

df_tgn['country'] = df_tgn['address'].astype(str).str.split(',').str[-1]


  0%|          | 0/835649 [00:00<?, ?it/s]

KeyboardInterrupt: 

3                     Yemen
4      United Arab Emirates
5      United Arab Emirates
6      United Arab Emirates
7      United Arab Emirates
8      United Arab Emirates
9                     Japan
10                   Israel
12                     Iran
13                    India
Name: address, dtype: object

In [83]:
df_tgn = df_tgn.head(30)

In [23]:
tqdm.pandas()

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="Nancy Amandi", timeout= 10)
rgeocode = RateLimiter(geolocator.reverse, min_delay_seconds=0.1)
df_tgn["location"] = df_tgn["coordinates"].progress_apply(rgeocode)
print(df.head(10))

NameError: name 'tqdm' is not defined

In [21]:

df_tgn.head(1)

NameError: name 'df_tgn' is not defined

In [20]:
# from geopy.geocoders import Nominatim

# geolocator = Nominatim(user_agent="geopy.geocoders.Nominatim")
# location = geolocator.reverse("52.509669, 13.376294")
#

from pprint import pprint
pprint(location.raw)
print(location.address)
print(location.raw['address']['country'])
print(location.raw['address']['city'])

{'address': {'ISO3166-2-lvl4': 'DE-BE',
             'borough': 'Mitte',
             'city': 'Berlin',
             'country': 'Deutschland',
             'country_code': 'de',
             'house_number': '11',
             'postcode': '10785',
             'road': 'Potsdamer Platz',
             'suburb': 'Tiergarten'},
 'addresstype': 'building',
 'boundingbox': ['52.5085787', '52.5090046', '13.3753432', '13.3758898'],
 'class': 'building',
 'display_name': '11, Potsdamer Platz, Tiergarten, Mitte, Berlin, 10785, '
                 'Deutschland',
 'importance': 9.99999999995449e-06,
 'lat': '52.5088708',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
            'http://osm.org/copyright',
 'lon': '13.375710625405464',
 'name': '',
 'osm_id': 216503834,
 'osm_type': 'way',
 'place_id': 128482208,
 'place_rank': 30,
 'type': 'commercial'}
11, Potsdamer Platz, Tiergarten, Mitte, Berlin, 10785, Deutschland
Deutschland
Berlin


In [49]:
df =df_tgn.head(1)

In [25]:
df['col_3'] = df.apply(lambda x: pandas_locater(x['lat.value'], x['long.value']), axis=1)

# df.apply(lambda x: pandas_locater(x., x.), axis=1)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=13.786&lon=46.141&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [11]:
from geopy.geocoders import Nominatim


11, Potsdamer Platz, Tiergarten, Mitte, Berlin, 10785, Deutschland


In [8]:
!pip install geopy

  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached geographiclib-2.0-py3-none-any.whl.metadata (1.4 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached geographiclib-2.0-py3-none-any.whl (40 kB)
